<a href="https://colab.research.google.com/github/eakubrakova/MFTI_2023/blob/main/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
covid_data = pd.read_csv('/content/covid_data.csv')
display(covid_data.head())
vaccinations_data = pd.read_csv('/content/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations',
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1
В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.

2
В таблицах не совпадает число стран, а иногда и их названия.

При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.

3
Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

In [ ]:
"""В таблице covid_data:

1) Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам.
 Тем самым переходим от данных по регионам к данным по странам:"""

covid_data = covid_data.groupby(
    ['date', 'country'],
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

"""Преобразуем даты в формат datetime с помощью функции pd.to_datetime():"""

covid_data['date'] = pd.to_datetime(covid_data['date'])

"""Создадим признак больных на данный момент (active). Для этого
 вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:"""

covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

"""Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей.
 Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём
 группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():"""

covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

"""В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:"""

vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

period = covid_data['date'].sort_values()

period2 = vaccinations_data['date'].sort_values()
# объединение таблиц
covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')

# создание признаков death_rate и recover_rate
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100

#Какова максимальная летальность в США (United States) за весь период?
#Ответ округлите до второго знака после запятой.
round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)

# Чему равен средний процент выздоровевших в России (Russia)?
#Ответ округлите до второго знака после запятой.
round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)


67.06